In [33]:
import numpy as np 
import pandas as pd 
import scipy as sps 
from scipy.stats import f as F, t as T

np.set_printoptions(suppress=True, precision=3)
pd.set_option('precision', 2)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [34]:
def regression(X, b, Y, end='\n'):
    if isinstance(X, pd.DataFrame): X = X.values 
    if isinstance(Y, pd.DataFrame) or isinstance(Y, pd.Series): Y = Y.values 

    for x, y in zip(X, Y):
        print('y = ' + ' + '.join([f'{b_.round(2)}*{x_.round(2)}' for (x_, b_) in zip(x, b)]) + f' = {y.round(3)}', end=end)

$y = b_0 + b_1 x_1 + b_2 x_2 + b_3 x_3 + b_{12} x_1 x_2 + b_{13} x_1 x_3 + b_{23} x_2 x_3 + b_{123} x_1 x_2 x_3 + b_{11} x^{2}_1 + b_{22} x^{2}_2 + b_{33} x^{2}_3$

In [35]:
# x1, x2, x3 = (-2, 5), (0, 3), (-9, 10) 
x1, x2, x3 = (-4, 4), (-10, 4), (-5, 6) 

columns = []
for c1 in ['x1', 'x2', 'x3']: columns += [(c1, 'min'), (c1, 'max')]
    
df = pd.DataFrame(data=[[*x1, *x2, *x3]], columns=columns, index=['105'])
df.columns = pd.MultiIndex.from_tuples(columns)
df

x1      x2      x3    
    min max min max min max
105  -4   4 -10   4  -5   6

In [36]:
x_cp_max = sum(map(max, [x1, x2, x3])) / 3
x_cp_min = sum(map(min, [x1, x2, x3])) / 3
y_imax, y_imin = 200 + x_cp_max, 200 + x_cp_min

Для того щоб отримати значення факторів для зоряних точок і для нульової точки, необхідно використовувати формули з лабораторної роботи № 1, нагадаємо їх:

In [37]:
x0i = np.array([np.mean(xi) for xi in [x1, x2, x3]])
dxi = np.array(list(map(max, [x1, x2, x3]))) - x0i 

p = 0
k = 3 
N = 15 

l = np.sqrt(np.sqrt(2 ** (k - p - 2) * (2 ** (k - p) + 2 * k + 1)) - 2 ** (k - p - 1))

Матриця планування експерименту для ОЦКП при k=3 із нормованими значеннями факторів наведена нижче

In [38]:
from sklearn.preprocessing import PolynomialFeatures

X = np.array([
    [-1, -1, -1],
    [-1, -1, +1],
    [-1, +1, -1],
    [-1, +1, +1],
    [+1, -1, -1],
    [+1, -1, +1],
    [+1, +1, -1],
    [+1, +1, +1],
    *np.zeros((7, 3)).tolist()
])

inds = np.diag_indices(3)
inds = inds[0] + 8 + range(3), inds[1]
X[inds] -= l 
inds = inds[0] + 1, inds[1]
X[inds] += l


poly_x = PolynomialFeatures(3, include_bias=False, interaction_only=True).fit_transform(X)

m = 3
y = np.random.randint(y_imin, y_imax, size=(15, m))
                      
poly_x = np.column_stack([poly_x, poly_x[:, :3] ** 2, y, y.mean(axis=1, keepdims=True)])
MPnorm = pd.DataFrame(poly_x, columns=['x1', 'x2', 'x3', 'x1x2', 'x1x3', 'x2x3', 'x1x2x3', 'x1^2', 'x2^2', 'x3^3', 'y1', 'y2', 'y3', 'y'])
MPnorm

,x1,x2,x3,x1x2,x1x3,x2x3,x1x2x3,x1^2,x2^2,x3^3,y1,y2,y3,y
0,-1.00,-1.00,-1.00,1.0,1.0,1.0,-1.0,1.00,1.00,1.00,202.0,196.0,195.0,197.67
1,-1.00,-1.00,1.00,1.0,-1.0,-1.0,1.0,1.00,1.00,1.00,197.0,199.0,202.0,199.33
2,-1.00,1.00,-1.00,-1.0,1.0,-1.0,1.0,1.00,1.00,1.00,193.0,196.0,202.0,197.00
3,-1.00,1.00,1.00,-1.0,-1.0,1.0,-1.0,1.00,1.00,1.00,196.0,195.0,194.0,195.00
4,1.00,-1.00,-1.00,-1.0,-1.0,1.0,1.0,1.00,1.00,1.00,199.0,194.0,199.0,197.33
5,1.00,-1.00,1.00,-1.0,1.0,-1.0,-1.0,1.00,1.00,1.00,195.0,195.0,196.0,195.33
6,1.00,1.00,-1.00,1.0,-1.0,-1.0,-1.0,1.00,1.00,1.00,195.0,196.0,196.0,195.67
7,1.00,1.00,1.00,1.0,1.0,1.0,1.0,1.00,1.00,1.00,195.0,193.0,195.0,194.33
8,-1.22,0.00,0.00,-0.0,-0.0,0.0,-0.0,1.48,0.00,0.00,194.0,203.0,197.0,198.00
9,1.22,0.00,0.00,0.0,0.0,0.0,0.0,1.48,0.00,0.00,198.0,200.0,195.0,197.67


Матриця планування експерименту для ОЦКП при k=3 із натуралізованими значеннями факторів має вигляд:

In [39]:
X = np.array([
    [-1, -1, -1],
    [-1, -1, +1],
    [-1, +1, -1],
    [-1, +1, +1],
    [+1, -1, -1],
    [+1, -1, +1],
    [+1, +1, -1],
    [+1, +1, +1],

])

xmap = [x1, x2, x3]
for i in range(X.shape[1]):
    X[:, i] = np.where(X[:, i] == -1, *xmap[i])

    
x = np.repeat([x0i], 7, axis=0)

inds = np.diag_indices(3)
inds = inds[0] + range(3), inds[1]
x[inds] -= l * dxi 
inds = inds[0] + 1, inds[1]
x[inds] += l * dxi 

X = np.row_stack([X, x])

poly_x = PolynomialFeatures(3, include_bias=False, interaction_only=True).fit_transform(X)
poly_x = np.column_stack([poly_x, poly_x[:, :3] ** 2, y, y.mean(axis=1, keepdims=True)])
MPnatur = pd.DataFrame(poly_x, columns=['x1', 'x2', 'x3', 'x1x2', 'x1x3', 'x2x3', 'x1x2x3', 'x1^2', 'x2^2', 'x3^3', 'y1', 'y2', 'y3', 'y'])
MPnatur

,x1,x2,x3,x1x2,x1x3,x2x3,x1x2x3,x1^2,x2^2,x3^3,y1,y2,y3,y
0,-4.00,-10.00,-5.00,40.00,20.00,50.00,-200.00,16.00,100.00,25.00,202.0,196.0,195.0,197.67
1,-4.00,-10.00,6.00,40.00,-24.00,-60.00,240.00,16.00,100.00,36.00,197.0,199.0,202.0,199.33
2,-4.00,4.00,-5.00,-16.00,20.00,-20.00,80.00,16.00,16.00,25.00,193.0,196.0,202.0,197.00
3,-4.00,4.00,6.00,-16.00,-24.00,24.00,-96.00,16.00,16.00,36.00,196.0,195.0,194.0,195.00
4,4.00,-10.00,-5.00,-40.00,-20.00,50.00,200.00,16.00,100.00,25.00,199.0,194.0,199.0,197.33
5,4.00,-10.00,6.00,-40.00,24.00,-60.00,-240.00,16.00,100.00,36.00,195.0,195.0,196.0,195.33
6,4.00,4.00,-5.00,16.00,-20.00,-20.00,-80.00,16.00,16.00,25.00,195.0,196.0,196.0,195.67
7,4.00,4.00,6.00,16.00,24.00,24.00,96.00,16.00,16.00,36.00,195.0,193.0,195.0,194.33
8,-4.86,-3.00,0.50,14.58,-2.43,-1.50,7.29,23.64,9.00,0.25,194.0,203.0,197.0,198.00
9,4.86,-3.00,0.50,-14.58,2.43,-1.50,-7.29,23.64,9.00,0.25,198.0,200.0,195.0,197.67


In [42]:
A = MPnatur.iloc[:, :-4].copy().values
A = np.insert(A, 0, 1, axis=1)

b = MPnatur['y'].copy().values

В цей раз ми маємо не квадратну($15x10$) матрицю, а отже розкладемо матрицю на дві *np.linalg.qr(A)*, тобто $QxR = A$

тепер наше рівняння $\|Ax-b\|_2$ прийме вигляд $\|Rx-Q^Tb\|_2$

[приклад](https://andreask.cs.illinois.edu/cs357-s15/public/demos/06-qr-applications/Solving%20Least-Squares%20Problems.html)

In [43]:
np.linalg.solve(A.T@A, A.T@b)

array([199.622,  -0.119,  -0.307,  -0.124,   0.009,  -0.006,  -0.01 ,
         0.004,  -0.093,  -0.014,  -0.06 ])

In [44]:
Q, R = np.linalg.qr(A)

x = sps.linalg.solve_triangular(R, Q.T@b, lower=False) 
x

array([199.622,  -0.119,  -0.307,  -0.124,   0.009,  -0.006,  -0.01 ,
         0.004,  -0.093,  -0.014,  -0.06 ])

In [45]:
regression(MPnatur.iloc[:, :-4], x, MPnatur['y'], end='\n\n')

y = 199.62*-4.0 + -0.12*-10.0 + -0.31*-5.0 + -0.12*40.0 + 0.01*20.0 + -0.01*50.0 + -0.01*-200.0 + 0.0*16.0 + -0.09*100.0 + -0.01*25.0 = 197.667

y = 199.62*-4.0 + -0.12*-10.0 + -0.31*6.0 + -0.12*40.0 + 0.01*-24.0 + -0.01*-60.0 + -0.01*240.0 + 0.0*16.0 + -0.09*100.0 + -0.01*36.0 = 199.333

y = 199.62*-4.0 + -0.12*4.0 + -0.31*-5.0 + -0.12*-16.0 + 0.01*20.0 + -0.01*-20.0 + -0.01*80.0 + 0.0*16.0 + -0.09*16.0 + -0.01*25.0 = 197.0

y = 199.62*-4.0 + -0.12*4.0 + -0.31*6.0 + -0.12*-16.0 + 0.01*-24.0 + -0.01*24.0 + -0.01*-96.0 + 0.0*16.0 + -0.09*16.0 + -0.01*36.0 = 195.0

y = 199.62*4.0 + -0.12*-10.0 + -0.31*-5.0 + -0.12*-40.0 + 0.01*-20.0 + -0.01*50.0 + -0.01*200.0 + 0.0*16.0 + -0.09*100.0 + -0.01*25.0 = 197.333

y = 199.62*4.0 + -0.12*-10.0 + -0.31*6.0 + -0.12*-40.0 + 0.01*24.0 + -0.01*-60.0 + -0.01*-240.0 + 0.0*16.0 + -0.09*100.0 + -0.01*36.0 = 195.333

y = 199.62*4.0 + -0.12*4.0 + -0.31*-5.0 + -0.12*16.0 + 0.01*-20.0 + -0.01*-20.0 + -0.01*-80.0 + 0.0*16.0 + -0.09*16.0 + -0.01*25.0 = 195.667

Оскільки отримані значення з невеликим відхиленням збігаються із середніми значеннями yi, то значення коефіцієнтів рівняння

In [46]:
A @ x - b # похибки 

array([ 0.728, -0.06 , -0.53 , -1.319,  1.081,  0.293, -0.177, -0.966,
        0.903, -0.259, -1.749,  2.392, -0.976,  1.619, -0.979])

Далі проведемо статистичні перевірки – на однорідність дисперсії за критерієм Кохрена, на нуль-гіпотезу за критерієм Стьюдента і на адекватність моделі за критерієм Фішера.

**Перевірка однорідності дисперсії за критерієм Кохрена**

In [47]:
def cochran_q(X, p=0.95): 
    """ Перевірка однорідності дисперсії за критерієм Кохрена """
    
    N, m = X.shape 

    Xvar = ((X - X.mean(axis=1, keepdims=True)) ** 2).sum(axis=1) / m # Xvar = X.var(axis=1)
    G = Xvar.max() / Xvar.sum()
    
    f1, f2 = m - 1, N 
    q = 1 - p # Рівень значимості
    
    Gf = F.sf(q / f2, f1, (f2 - 1) * f1) # isf 
    Gf = Gf / (Gf + f1 - 1)
    
    columns = ['f1', 'f2', 'q', 'G', 'Gt', 'uniform']
    return pd.DataFrame([[f1, f2, q, G, Gf, G < Gf]], columns=columns)


log = cochran_q(MPnatur[['y1', 'y2', 'y3']].values) # m = 2, N = 8 
if log['uniform'].item(): 
    print(f'G < Gt, {log["G"].item():.3f} < {log["Gt"].item():.3f} - Дисперсія однорідна.')
else: 
    print(f'G > Gt, {log["G"].item():.3f} > {log["Gt"].item():.3f} - Дисперсія неоднорідна.')

G < Gt, 0.221 < 0.499 - Дисперсія однорідна.


**Перевірка нуль-гіпотези за критерієм Стьюдента**

In [147]:
def t_test(X, Y, p=0.95): 
    """ Значимість коефіцієнтів регресії згідно критерію Стьюдента
        :return: nd.array - індекси значимих коефіцієнтів рівняння регресії
    """
    
    N, m = Y.shape
    f1, f2 = m - 1, N 
    f3 = f1 * f2 
    q = 1 - p 
    
    S_sqr = Y.var(axis=1).mean() / (m * N)
    betas = (X.T * Y.mean(axis=1)).mean(axis=1)
    t = abs(betas) / np.sqrt(S_sqr)
    
    # first variant 
    T_st = abs(T.ppf(q / 2, f3)) # 2.042(f3 = 30) in lab 5 
    ind_in = np.argwhere(t > T_st).flatten()
    ind_out = np.argwhere(t < T_st).flatten()
    

    columns = ['f1', 'f2', 'f3', 'q', 'T', 'Tt', 'd', 'significant_inds', 'not']
    return pd.DataFrame([[f1, f2, f3, q, t.round(2).tolist(), T_st, len(ind_in), ind_in.tolist(), ind_out.tolist()]], columns=columns)               


A = MPnorm.iloc[:, :-4].values
A = np.insert(A, 0, 1, axis=1)
log = t_test(A, MPnorm[['y1', 'y2', 'y3']].values)
log

,f1,f2,f3,q,T,Tt,d,significant_inds,not
0,2,15,30,0.05,"[644.51, 1.47, 3.78, 2.03, 0.51, 0.65, 0.65, 0...",2.04,5,"[0, 2, 8, 9, 10]","[1, 3, 4, 5, 6, 7]"


In [148]:
inds = log['significant_inds'].item()
regression(A[:, inds], x[inds], A[:, inds]@x[inds])

y = 199.62*1.0 + -0.31*-1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.762
y = 199.62*1.0 + -0.31*-1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.762
y = 199.62*1.0 + -0.31*1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.147
y = 199.62*1.0 + -0.31*1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.147
y = 199.62*1.0 + -0.31*-1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.762
y = 199.62*1.0 + -0.31*-1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.762
y = 199.62*1.0 + -0.31*1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.147
y = 199.62*1.0 + -0.31*1.0 + -0.09*1.0 + -0.01*1.0 + -0.06*1.0 = 199.147
y = 199.62*1.0 + -0.31*0.0 + -0.09*1.48 + -0.01*0.0 + -0.06*0.0 = 199.485
y = 199.62*1.0 + -0.31*0.0 + -0.09*1.48 + -0.01*0.0 + -0.06*0.0 = 199.485
y = 199.62*1.0 + -0.31*-1.22 + -0.09*0.0 + -0.01*1.48 + -0.06*0.0 = 199.975
y = 199.62*1.0 + -0.31*1.22 + -0.09*0.0 + -0.01*1.48 + -0.06*0.0 = 199.227
y = 199.62*1.0 + -0.31*0.0 + -0.09*0.0 + -0.01*0.0 + -0.06*1.48 = 199.533
y = 199.62*1.0 + -0.31*0.0 + -0.09*0.0 

In [149]:
(A[:, inds]@x[inds] - MPnorm['y']).round(3).values # похибки

array([ 2.095,  0.429,  2.147,  4.147,  2.429,  4.429,  3.48 ,  4.814,
        1.485,  1.818, -3.025,  4.227, -0.134,  4.533, -1.711])

**Перевірка адекватності моделі по критерію Фішера**

In [150]:
def F_test(y1, y2, d):
    """ 
    Адекватність моделі за F-критерієм Фішера, який дорівнює відношенню дисперсії
    адекватності до дисперсії відтворюваності:
    """
    N, m = y1.shape
    f1, f2 = m - 1, N 
    f3 = f1 * f2 
    f4 = N - d
    q = 1 - p 
    S_sqr_ad = (m / f4) * ((y2 - y1.mean(axis=1)) ** 2).mean()
    S_sqr = y1.var(axis=1).mean()
    F_val = S_sqr_ad / S_sqr
    
    F_t = 3.1
    
    columns = ['f1', 'f2', 'f3', 'f4', 'q', 'F', 'Ft', 'adequate']
    return pd.DataFrame([[f1, f2, f3, f4, q, F_val, F_t, F_val < F_t]], columns=columns)
    
    
d = log['d'].item() # кількість значимих коефіцієнтів 
y = MPnorm[['y1', 'y2', 'y3']].values
yn = A[:, inds]@x[inds]

F_test(y, yn, d)
log = F_test(y, yn, d)

message = lambda x1, x2: f'Fp {x1} Ft, {log["F"].item():.3f} {x1} {log["Ft"].item():.3f}\nрівняння регресії {x2}адекватнe оригіналу при рівні значимості 0.05'
if log['adequate'].item():
    print(message('<', ''))
else:
    print(message('>', 'не'))

Fp < Ft, 0.678 < 3.100
рівняння регресії адекватнe оригіналу при рівні значимості 0.05
